In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pipe = pipeline(task="zero-shot-classification", model="cointegrated/rubert-base-cased-nli-threeway", framework='pt', device=device)
device

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

device(type='cpu')

In [4]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import pandas as pd

data = pd.DataFrame({'text': ['Мне нужно срочно починить телефон', 'Мне не нужен новый телефон']})
data

,text
0,Мне нужно срочно починить телефон
1,Мне не нужен новый телефон


In [6]:
from datasets import Dataset

data = Dataset.from_pandas(data)
data

Dataset({
    features: ['text'],
    num_rows: 2
})

In [7]:
pred_y = pipe(
    data['text'],
    candidate_labels=["срочно", "не срочно", "телефон", "планшет", "компьютер", 'холодильник'],
)
pred_y

[{'sequence': 'Мне нужно срочно починить телефон',
  'labels': ['срочно',
   'телефон',
   'не срочно',
   'холодильник',
   'компьютер',
   'планшет'],
  'scores': [0.7179533243179321,
   0.2631394565105438,
   0.006201811600476503,
   0.005608343984931707,
   0.003835140261799097,
   0.003261963138356805]},
 {'sequence': 'Мне не нужен новый телефон',
  'labels': ['не срочно',
   'телефон',
   'срочно',
   'холодильник',
   'планшет',
   'компьютер'],
  'scores': [0.5978946089744568,
   0.2704094350337982,
   0.08599814027547836,
   0.022706735879182816,
   0.011677985079586506,
   0.011313091032207012]}]

In [8]:
pred_y = [pred['labels'][0] for pred in pred_y]
pred_y

['срочно', 'не срочно']

In [21]:
# если не хватает памяти
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

batched_data = DataLoader(data, batch_size=64, shuffle=False)
pred_y = []
for batch in tqdm(batched_data):
  pred_cur = pipe(
      batch['text'],
      candidate_labels=["срочно", "не срочно", "телефон", "планшет", "компьютер", 'холодильник']
  )
  pred_y += [pred['labels'][0] for pred in pred_cur]
pred_y

# batched_data = DataLoader(data['text'], batch_size=64, shuffle=False)
# pred_y = []
# for batch in tqdm(batched_data):
#   pred_cur = pipe(
#       batch,
#       candidate_labels=["срочно", "не срочно", "телефон", "планшет", "компьютер", 'холодильник']
#   )
#   pred_y += [pred['labels'][0] for pred in pred_cur]

  0%|          | 0/1 [00:00<?, ?it/s]

['срочно', 'не срочно']